In [18]:
import os
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import mne # reads edf format
import glob
import random
import re
import wfdb # waveform database library wfdb.rdann reads annotation of physiology annotated data in ECG, EEG etc

In [2]:
channel_labels = ['FP1-F7', 'F7-T7', 'T7-P7', 'P7-O1', 'FP1-F3', 'F3-C3', 'C3-P3','P3-O1',
                  'FP2-F4', 'F4-C4', 'C4-P4', 'P4-O2', 'FP2-F8', 'F8-T8', 'T8-P8', 'P8-O2',
                  'FZ-CZ', 'CZ-PZ']

In [3]:
path = r'data\raw_data\chb-mit-scalp-eeg-database-1.0.0'
data_folders = sorted(glob.glob(os.path.join(path, '*[0-9]')))

def file_id(folder):
    return [ name[-2:] for name in [file.rsplit('\\',2)[-1] for file in folder]]
print("ID: ")
print(file_id(data_folders))

ID: 
['01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '21', '22', '23', '24']


In [4]:
# seizure  marking is in name of edf file, not a separate file
# split files for training and testing
train_test_ratio = 0.8
random.seed(80)
train_folders = sorted(random.sample(data_folders, round(len(data_folders) * train_test_ratio)))
test_folders = sorted([ file for file in data_folders if file not in train_folders])
print(f"Training files ID: {file_id(train_folders)}")
print(f"Test files ID: {file_id(test_folders)}")



Training files ID: ['01', '02', '04', '07', '08', '09', '10', '12', '13', '14', '15', '16', '17', '18', '19', '20', '22', '23', '24']
Test files ID: ['03', '05', '06', '11', '21']


In [5]:
# Retrieve edf files

train_files = [ file for folder in train_folders for file in glob.glob(folder+'/*.edf')]
test_files = [ file for folder in test_folders for file in glob.glob(folder+'/*.edf')]

print(f"Train_files contains {len(train_files)} files")
print(f"Test_files contains {len(test_files)} files")

Train_files contains 523 files
Test_files contains 163 files


In [13]:
# MEG and EEG library (mne) reads neurophysiological data
sample_edf = mne.io.read_raw_edf(train_files[0], preload=False) #110-15 seconds
# dir(sample_edf)  # show all attributes and methods
sample_edf.info

Extracting EDF parameters from C:\Users\mspla\Documents\repos\py_basic_310_Au24\seizurePy3.9\data\raw_data\chb-mit-scalp-eeg-database-1.0.0\chb01\chb01_01.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...


C:\Users\mspla\AppData\Local\Temp\ipykernel_12772\1770090686.py:2: RuntimeWarning: Channel names are not unique, found duplicates for: {'T8-P8'}. Applying running numbers for duplicates.
  sample_edf = mne.io.read_raw_edf(train_files[0], preload=False) #110-15 seconds


<Info | 8 non-empty values
 bads: []
 ch_names: FP1-F7, F7-T7, T7-P7, P7-O1, FP1-F3, F3-C3, C3-P3, P3-O1, ...
 chs: 23 EEG
 custom_ref_applied: False
 highpass: 0.0 Hz
 lowpass: 128.0 Hz
 meas_date: 2076-11-06 11:42:54 UTC
 nchan: 23
 projs: []
 sfreq: 256.0 Hz
 subject_info: 1 item (dict)
>

In [33]:
'''
Reading EEG data, modified from cell 60 of notebook by Masahiro Gotoh   [available at https://www.kaggle.com/code/hemangjindal/chb-mit-eeg-dataset-my-notebook#CHB-MIT-eeg-dataset-seizure-detection-demo]
'''
window_time = 10  # segment size in second
step_time = 5     # Step size in second
sampling_rate = 16  # EEGb Sampling rate (Hz)
no_seizure_portion = 0.01    # proportion of non seizure
counter = 0
preictal_label = 1
interictal_label = 0


def read_edf(folder):
    for file in folder:
        edf_data = mne.io.read_raw_edf(file, preload=False)
        edf_labels = edf_data.ch_names
        # check all channel labels appear in edf_labels
        if sum([any([0 if re.match(c, l) is None else 1 for l in edf_labels]) for c in channel_labels]) == len(channel_labels):
            segment_ = 10
            stride = 5
            sampling_freq = 1/(edf_data.times[1]-edf_data.times[0])
            window_size = sampling_freq * window_time
            window_stride = sampling_freq * step_time

            # identity EEG signal with seizure. 'Seizure' appended to file name, and marked at time point of seizure activity inside file.
            # has seizure marks seizure/non-seizure as (1/0) for every data point
            has_seizure = np.zeros((edf_data.n_times,))
            if os.path.exists(file + '.seizures'):
                has_annotation = wfdb.rdann(file, 'seizures')
                # has_annotation.sample e.g [10,20, 300,400] means marked seizure from sample 10-20 , and 300-400
                for idx in range(int(has_annotation.sample.size / 2)):
                    has_seizure[has_annotation.sample[idx * 2]:has_annotation.sample[idx * 2 + 1]] = 1

            # ensure remains integer
            window_size = int(window_size)
            window_stride = int(window_stride)
            #creawte seizure index, and calculate proportion of signal which shows seizures
            is_seizure_idx = np.array([has_seizure[idx * window_stride:idx * window_stride + window_size].sum() / window_size for idx in range((edf_data.n_times - window_size) // window_stride)])














#
#
# # Derived parameters
# window_size = window_size_sec * sampling_rate  # Convert to samples
# step_size = step_size_sec * sampling_rate  # Convert to samples
#
# # Function to load and prepare data
# def load_and_prepare_data(csv_file):
#     """
#     Load the merged CSV data and prepare it for LSTM model training.
#     """
#     data = pd.read_csv(csv_file)
#
#     # Normalize the EEG data (Min-Max scaling)
#     scaler = MinMaxScaler()
#     eeg_data = data[eeg_channels].values
#     eeg_data_scaled = scaler.fit_transform(eeg_data)
#
#     # Get the labels
#     labels = data['Label'].values
#
#     # Create sliding windows
#     windows, window_labels = create_windows(eeg_data_scaled, labels, window_size, step_size)
#
#     return windows, window_labels, scaler
#
# # Function to create windows from the data
# def create_windows(data, labels, window_size, step_size):
#     """
#     Create overlapping windows from EEG data and corresponding labels.
#     """
#     windows = []
#     window_labels = []
#
#     for start in range(0, len(data) - window_size + 1, step_size):
#         end = start + window_size
#         window = data[start:end, :]
#         label = labels[start:end]
#
#         # Assign label based on majority class in the window
#         window_label = preictal_label if np.sum(label == preictal_label) > len(label) // 2 else interictal_label
#         windows.append(window)
#         window_labels.append(window_label)
#
#     return np.array(windows), np.array(window_labels)
#
# # Function to build the LSTM model
# def build_lstm_model(input_shape):
#     """
#     Build and compile an LSTM model.
#     """
#     print('Input shape-',input_shape)
#     model = Sequential()
#     model.add(LSTM(units=64, return_sequences=True, input_shape=input_shape))
#     model.add(Dropout(0.2))
#     model.add(LSTM(units=32, return_sequences=False))
#     model.add(Dropout(0.2))
#     model.add(Dense(units=1, activation='sigmoid'))  # Output layer with sigmoid for binary classification
#     model.compile(optimizer=Adam(learning_rate=0.001), loss='binary_crossentropy', metrics=['accuracy'])
#     return model
#
# # Function to train the LSTM model
# def train_lstm_model(windows, window_labels):
#     """
#     Train the LSTM model using the provided windows and labels.
#     """
#     # Split data into training and testing sets
#     X_train, X_test, y_train, y_test = train_test_split(windows, window_labels, test_size=0.2, random_state=42)
#
#     # Build and train the model
#     model = build_lstm_model((X_train.shape[1], X_train.shape[2]))
#     model.fit(X_train, y_train, epochs=10, batch_size=64, validation_split=0.2)
#
#     # Evaluate the model on the test set
#     y_pred = model.predict(X_test)
#     y_pred = (y_pred > 0.5).astype(int)  # Convert probabilities to binary labels
#
#     # Print classification report
#     print("Classification Report on Test Data:")
#     print(classification_report(y_test, y_pred))
#
#     # Return the trained model
#     return model, scaler
#
# # Real-time prediction function
# def real_time_prediction(model, scaler, window_size, interval=30):
#     """
#     Simulate real-time predictions every `interval` seconds using the trained LSTM model.
#     """
#     print(f"Starting real-time prediction... (Interval: {interval}s)")
#
#     # Simulate receiving new data (use the merged data as a placeholder)
#     while True:
#         # Simulate receiving data
#         # Replace this with real-time data fetching process
#         # For now, we assume the data is available in 'merged_df'
#         # Here we can simulate a chunk of data for prediction
#         simulated_data = np.random.rand(window_size, len(eeg_channels))  # Simulating new data
#
#         # Normalize the new data
#         scaled_data = scaler.transform(simulated_data)
#
#         # Reshape data to match LSTM input shape
#         X_input = scaled_data.reshape(1, window_size, len(eeg_channels))
#
#         # Predict the probability of seizure (preictal)
#         prediction = model.predict(X_input)
#         print(f"Predicted probability of seizure: {prediction[0][0]:.4f}")
#
#         # Sleep for the specified interval (simulate 30 seconds)
#         time.sleep(interval)
#
# # Main function
# if __name__ == "__main__":
#     # Load the merged data
#     csv_file = '/kaggle/working/csv_files/merged_data.csv'
#     windows, window_labels, scaler = load_and_prepare_data(csv_file)
#
#     # Train the LSTM model and evaluate on the test data
#     model, scaler = train_lstm_model(windows, window_labels)
#
#     # Save the model for later use
#     model.save('/kaggle/working/lstm_model.h5')
#     print("Model saved to /kaggle/working/lstm_model.h5")
#
#     # Start real-time prediction
#     real_time_prediction(model, scaler, window_size, interval=30)

In [34]:
read_edf(train_files)

Extracting EDF parameters from C:\Users\mspla\Documents\repos\py_basic_310_Au24\seizurePy3.9\data\raw_data\chb-mit-scalp-eeg-database-1.0.0\chb01\chb01_01.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 921599  =      0.000 ...  3599.996 secs...


C:\Users\mspla\AppData\Local\Temp\ipykernel_12772\2212051881.py:15: RuntimeWarning: Channel names are not unique, found duplicates for: {'T8-P8'}. Applying running numbers for duplicates.
  edf_data = mne.io.read_raw_edf(file, preload=True)


Extracting EDF parameters from C:\Users\mspla\Documents\repos\py_basic_310_Au24\seizurePy3.9\data\raw_data\chb-mit-scalp-eeg-database-1.0.0\chb01\chb01_02.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 921599  =      0.000 ...  3599.996 secs...


C:\Users\mspla\AppData\Local\Temp\ipykernel_12772\2212051881.py:15: RuntimeWarning: Channel names are not unique, found duplicates for: {'T8-P8'}. Applying running numbers for duplicates.
  edf_data = mne.io.read_raw_edf(file, preload=True)


Extracting EDF parameters from C:\Users\mspla\Documents\repos\py_basic_310_Au24\seizurePy3.9\data\raw_data\chb-mit-scalp-eeg-database-1.0.0\chb01\chb01_03.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 921599  =      0.000 ...  3599.996 secs...


...

C:\Users\mspla\AppData\Local\Temp\ipykernel_12772\2212051881.py:15: RuntimeWarning: Channel names are not unique, found duplicates for: {'T8-P8'}. Applying running numbers for duplicates.
  edf_data = mne.io.read_raw_edf(file, preload=True)


Extracting EDF parameters from C:\Users\mspla\Documents\repos\py_basic_310_Au24\seizurePy3.9\data\raw_data\chb-mit-scalp-eeg-database-1.0.0\chb24\chb24_21.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 921599  =      0.000 ...  3599.996 secs...


C:\Users\mspla\AppData\Local\Temp\ipykernel_12772\2212051881.py:15: RuntimeWarning: Channel names are not unique, found duplicates for: {'T8-P8'}. Applying running numbers for duplicates.
  edf_data = mne.io.read_raw_edf(file, preload=True)


Extracting EDF parameters from C:\Users\mspla\Documents\repos\py_basic_310_Au24\seizurePy3.9\data\raw_data\chb-mit-scalp-eeg-database-1.0.0\chb24\chb24_22.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 273151  =      0.000 ...  1066.996 secs...


C:\Users\mspla\AppData\Local\Temp\ipykernel_12772\2212051881.py:15: RuntimeWarning: Channel names are not unique, found duplicates for: {'T8-P8'}. Applying running numbers for duplicates.
  edf_data = mne.io.read_raw_edf(file, preload=True)


In [35]:
2+2

4

In [ ]:



# Configuration
csv_folder = '/kaggle/working/csv_files'  # Folder containing the merged CSV file
# eeg_channels = ['Channel_1', 'Channel_13', 'Channel_19', 'Channel_23']  # Channels to use
eeg_channels = ['Channel_5', 'Channel_15', 'Channel_3', 'Channel_2']  # Channels to use

window_size_sec = 5  # Window size in seconds
step_size_sec = 2 # Step size in seconds
sampling_rate = 16  # Sampling rate of EEG signals in Hz
preictal_label = 1
interictal_label = 0

# Derived parameters
window_size = window_size_sec * sampling_rate  # Convert to samples
step_size = step_size_sec * sampling_rate  # Convert to samples

# Function to load and prepare data
def load_and_prepare_data(csv_file):
    """
    Load the merged CSV data and prepare it for LSTM model training.
    """
    data = pd.read_csv(csv_file)

    # Normalize the EEG data (Min-Max scaling)
    scaler = MinMaxScaler()
    eeg_data = data[eeg_channels].values
    eeg_data_scaled = scaler.fit_transform(eeg_data)

    # Get the labels
    labels = data['Label'].values

    # Create sliding windows
    windows, window_labels = create_windows(eeg_data_scaled, labels, window_size, step_size)

    return windows, window_labels, scaler

# Function to create windows from the data
def create_windows(data, labels, window_size, step_size):
    """
    Create overlapping windows from EEG data and corresponding labels.
    """
    windows = []
    window_labels = []

    for start in range(0, len(data) - window_size + 1, step_size):
        end = start + window_size
        window = data[start:end, :]
        label = labels[start:end]

        # Assign label based on majority class in the window
        window_label = preictal_label if np.sum(label == preictal_label) > len(label) // 2 else interictal_label
        windows.append(window)
        window_labels.append(window_label)

    return np.array(windows), np.array(window_labels)

# Function to build the LSTM model
def build_lstm_model(input_shape):
    """
    Build and compile an LSTM model.
    """
    print('Input shape-',input_shape)
    model = Sequential()
    model.add(LSTM(units=64, return_sequences=True, input_shape=input_shape))
    model.add(Dropout(0.2))
    model.add(LSTM(units=32, return_sequences=False))
    model.add(Dropout(0.2))
    model.add(Dense(units=1, activation='sigmoid'))  # Output layer with sigmoid for binary classification
    model.compile(optimizer=Adam(learning_rate=0.001), loss='binary_crossentropy', metrics=['accuracy'])
    return model

# Function to train the LSTM model
def train_lstm_model(windows, window_labels):
    """
    Train the LSTM model using the provided windows and labels.
    """
    # Split data into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(windows, window_labels, test_size=0.2, random_state=42)

    # Build and train the model
    model = build_lstm_model((X_train.shape[1], X_train.shape[2]))
    model.fit(X_train, y_train, epochs=10, batch_size=64, validation_split=0.2)

    # Evaluate the model on the test set
    y_pred = model.predict(X_test)
    y_pred = (y_pred > 0.5).astype(int)  # Convert probabilities to binary labels

    # Print classification report
    print("Classification Report on Test Data:")
    print(classification_report(y_test, y_pred))

    # Return the trained model
    return model, scaler

# Real-time prediction function
def real_time_prediction(model, scaler, window_size, interval=30):
    """
    Simulate real-time predictions every `interval` seconds using the trained LSTM model.
    """
    print(f"Starting real-time prediction... (Interval: {interval}s)")

    # Simulate receiving new data (use the merged data as a placeholder)
    while True:
        # Simulate receiving data
        # Replace this with real-time data fetching process
        # For now, we assume the data is available in 'merged_df'
        # Here we can simulate a chunk of data for prediction
        simulated_data = np.random.rand(window_size, len(eeg_channels))  # Simulating new data

        # Normalize the new data
        scaled_data = scaler.transform(simulated_data)

        # Reshape data to match LSTM input shape
        X_input = scaled_data.reshape(1, window_size, len(eeg_channels))

        # Predict the probability of seizure (preictal)
        prediction = model.predict(X_input)
        print(f"Predicted probability of seizure: {prediction[0][0]:.4f}")

        # Sleep for the specified interval (simulate 30 seconds)
        time.sleep(interval)

# Main function
if __name__ == "__main__":
    # Load the merged data
    csv_file = '/kaggle/working/csv_files/merged_data.csv'
    windows, window_labels, scaler = load_and_prepare_data(csv_file)

    # Train the LSTM model and evaluate on the test data
    model, scaler = train_lstm_model(windows, window_labels)

    # Save the model for later use
    model.save('/kaggle/working/lstm_model.h5')
    print("Model saved to /kaggle/working/lstm_model.h5")

    # Start real-time prediction
    real_time_prediction(model, scaler, window_size, interval=30)